# Pandas Series: logika, rikiavimas, aritmetika, tekstas ir agregavimas

Šiame faile pateikiami praktiški `pandas.Series` pavyzdžiai:
loginiai operatoriai ir metodai, rikiavimas, aritmetiniai operatoriai ir metodai,
teksto (string) metodai, skaitinių reikšmių agregavimas ir kategorinių reikšmių suvestinės.


In [ ]:
import pandas as pd
import numpy as np

## 0. Pavyzdiniai duomenys

Žemiau pateikti keli paprasti `Series`, kurie imituoja produkto analitikos situacijas:
- pardavimai (vienetais)
- kaina (EUR)
- pajamos (EUR)
- produkto kategorija
- produkto pavadinimas

In [21]:
product = pd.Series(
    ["Protein Bar", "Sparkling Water", "Coffee Beans", "Granola", "USB Cable", "Notebook"],
    index=["P01", "P02", "P03", "P04", "P05", "P06"]
)

category = pd.Series(
    ["Snacks", "Beverages", "Beverages", "Snacks", "Electronics", "Stationery"],
    index=product.index
)

units_sold = pd.Series([120, 250, 80, 160, 300, 140], index=product.index)
price = pd.Series([2.49, 1.20, 8.90, 3.10, 5.50, 2.00], index=product.index)

revenue = units_sold * price

print("product:\n", product, "\n")
print("category:\n", category, "\n")
print("units_sold:\n", units_sold, "\n")
print("price:\n", price, "\n")
print("revenue:\n", revenue.round(2))

product:
 P01        Protein Bar
P02    Sparkling Water
P03       Coffee Beans
P04            Granola
P05          USB Cable
P06           Notebook
dtype: object 

category:
 P01         Snacks
P02      Beverages
P03      Beverages
P04         Snacks
P05    Electronics
P06     Stationery
dtype: object 

units_sold:
 P01    120
P02    250
P03     80
P04    160
P05    300
P06    140
dtype: int64 

price:
 P01    2.49
P02    1.20
P03    8.90
P04    3.10
P05    5.50
P06    2.00
dtype: float64 

revenue:
 P01     298.8
P02     300.0
P03     712.0
P04     496.0
P05    1650.0
P06     280.0
dtype: float64


In [22]:
print("revenue:\n", revenue.round(2))

revenue:
 P01     298.8
P02     300.0
P03     712.0
P04     496.0
P05    1650.0
P06     280.0
dtype: float64


## 1. Loginiai operatoriai ir metodai (logical operators & methods)

Pagrindinis principas: `Series` palyginimas grąžina boolean `Series`.

Dažniausiai naudojama:
- `>`, `>=`, `<`, `<=`, `==`, `!=`
- `&` (AND) ir `|` (OR) su skliaustais
- `~` (NOT)
- metodai: `between`, `isin`, `any`, `all`, `sum` (boolean atveju)

Geroji praktika:
- sudėtinėms sąlygoms visada naudoti skliaustus: `(cond1) & (cond2)`
- nenaudoti `and` / `or` su `Series` (tai sukelia klaidą)

In [23]:
high_units = units_sold > 150
print("high_units:\n", high_units, "\n")

# Boolean sum dažnai naudojamas skaičiuoti, kiek įrašų atitinka sąlygą
print("Kiek produktų parduota daugiau nei 150 vnt.:", high_units.sum())

high_units:
 P01    False
P02     True
P03    False
P04     True
P05     True
P06    False
dtype: bool 

Kiek produktų parduota daugiau nei 150 vnt.: 3


In [24]:
# Sudėtinė sąlyga: parduota daug ir pajamos pakankamai didelės
mask = (units_sold >= 150) & (revenue >= 300)

print("Atrankos kaukė:\n", mask, "\n")
print("Atrinkti produktai (ID):", list(mask[mask].index))
print("Atrinktos pajamos:\n", revenue[mask].round(2))

Atrankos kaukė:
 P01    False
P02     True
P03    False
P04     True
P05     True
P06    False
dtype: bool 

Atrinkti produktai (ID): ['P02', 'P04', 'P05']
Atrinktos pajamos:
 P02     300.0
P04     496.0
P05    1650.0
dtype: float64


### Dažna klaida: `and` / `or` vietoje `&` / `|`

`and` ir `or` yra Python operatoriai, skirti vienai boolean reikšmei.
`Series` atveju reikia naudoti `&` ir `|`.

Žemiau pateiktas pavyzdys rodomas kaip komentaras, nes jį vykdant būtų metama klaida.

In [27]:
#Žemiau pateiktas kodas yra dažna klaida:
# mask_wrong = (units_sold >= 150) and (revenue >= 300)
#ValueError: The truth value of a Series is ambiguous

mask_correct = (units_sold >= 150) & (revenue >= 300)
print(mask_correct)

P01    False
P02     True
P03    False
P04     True
P05     True
P06    False
dtype: bool


In [28]:
# between: patogus būdas intervalui
mid_units = units_sold.between(100, 200)  # įtraukiant ribas
print("between(100, 200):\n", mid_units, "\n")

# isin: priklausymas sąrašui
selected_categories = category.isin(["Snacks", "Electronics"])
print("isin(['Snacks','Electronics']):\n", selected_categories, "\n")

# any / all: greitam tikrinimui
print("Ar yra bent vienas produktas su pajamomis > 1000:", (revenue > 1000).any())
print("Ar visi produktai turi kainą > 0:", (price > 0).all())

between(100, 200):
 P01     True
P02    False
P03    False
P04     True
P05    False
P06     True
dtype: bool 

isin(['Snacks','Electronics']):
 P01     True
P02    False
P03    False
P04     True
P05     True
P06    False
dtype: bool 

Ar yra bent vienas produktas su pajamomis > 1000: True
Ar visi produktai turi kainą > 0: True


## 2. Series rikiavimas (sorting series)

Dažniausi rikiavimo būdai:
- `sort_values()` – rikiuoja pagal reikšmes
- `sort_index()` – rikiuoja pagal indeksą

Geroji praktika:
- rikiuojant aiškiai nurodyti `ascending=False`, kai reikia top reikšmių
- prieš lyginimą įsitikinti, kad tipas yra skaitinis (pvz., `object` gali rikiuotis kaip tekstas)

In [ ]:
print("Pajamos (originalas):\n", revenue.round(2), "\n")

print("Pajamos (mažėjimo tvarka):\n", revenue.sort_values(ascending=False).round(2), "\n")
print("Pajamos (didėjimo tvarka):\n", revenue.sort_values().round(2), "\n")



In [ ]:
print("Rikiavimas pagal indeksą:\n", revenue.sort_index().round(2))

## 3. Aritmetiniai operatoriai ir metodai (arithmetic operators & methods)

`Series` leidžia atlikti aritmetiką elementų lygiu.
Dažni operatoriai: `+`, `-`, `*`, `/`.

Dažni metodai:
- `add`, `sub`, `mul`, `div` – naudingi, kai reikia valdyti trūkstamas reikšmes arba suderinti indeksus
- `round` – aiškesniems rezultatams

Geroji praktika:
- remtis indeksu (pandas suderina reikšmes pagal indeksus)
- žinoti, kad nesutampantys indeksai duoda `NaN` (tai nėra klaida, tai signalas apie nesutapimą)

In [ ]:
# Paprastas pavyzdys: pajamos jau apskaičiuotos kaip units_sold * price
revenue2 = units_sold.mul(price)

print("revenue (operatorius) :\n", revenue.round(2), "\n")
print("revenue2 (metodas mul):\n", revenue2.round(2))

In [ ]:
# Pavyzdys su nesutampančiais indeksais (dažna reali situacija)
units_partial = pd.Series([120, 250, 80], index=["P01", "P02", "P03"])  # tik 3 produktai
price_full = price  # turi visus produktus

rev_misaligned = units_partial * price_full
print("rev_misaligned:\n", rev_misaligned.round(2))
print("Pastaba: NaN atsiranda ten, kur trūksta duomenų arba nesutampa indeksai.")

In [ ]:
# Metodai su fill_value padeda tvarkyti trūkstamas reikšmes
rev_safe = units_partial.mul(price_full, fill_value=0)

print("rev_safe:\n", rev_safe.round(2))
print("Pastaba: fill_value=0 reiškia, kad trūkstamas reikšmes laikinai laikomos 0 skaičiavimui.")

## 4. Teksto metodai (string methods)

Teksto apdorojimui naudojamas `.str` aksesorius.
Dažni metodai analitikoje:
- `lower`, `upper`, `strip`
- `contains` (paieška)
- `replace` (paprastas valymas)
- `len` (ilgis)

Geroji praktika:
- prieš `.str` veiksmus užtikrinti, kad reikšmės yra tekstinės (dažnai pasitaiko `NaN`)
- naudoti `na=False` su `contains`, kad trūkstamos reikšmės nesukeltų problemų

In [ ]:
names = pd.Series(["  Protein Bar  ", "Sparkling Water", None, "Coffee Beans", "USB Cable  "])

clean = names.str.strip().str.lower()

print("names:\n", names, "\n")
print("clean:\n", clean)

In [ ]:
# contains su na=False, kad None reikšmė būtų laikoma kaip neatitinkanti sąlygos
has_coffee = names.str.contains("Coffee", na=False)
print("has_coffee:\n", has_coffee, "\n")
print("Atitinkančios reikšmės:\n", names[has_coffee])

In [ ]:
# replace pavyzdys: standartizuojami produkto pavadinimai
raw_sku = pd.Series(["sku-001", "SKU-002", " sku-003 ", "Sku-004"])
sku = raw_sku.str.strip().str.upper().str.replace("SKU-", "SKU_", regex=False)

print("raw_sku:\n", raw_sku)
print("sku:\n", sku)

## 5. Skaitinių `Series` agregavimas (numeric series aggregation)

Dažni agregavimo metodai:
- `sum`, `mean`, `median`, `min`, `max`, `std`
- `quantile` – kvantiliai (pvz., P25, P50, P75)
- `describe` – greita santrauka

Geroji praktika:
- suprasti, kad agregavimas ignoruoja `NaN` (pagal nutylėjimą)
- aiškiai suapvalinti rodiklius, kai jie pateikiami kaip rezultatai

In [ ]:
print("Pajamų suma:", revenue.sum().round(2))
print("Pajamų vidurkis:", revenue.mean().round(2))
print("Pajamų mediana:", revenue.median())
print("Pajamų min / max:", revenue.min(), "/", revenue.max())
print("Pajamų std:", revenue.std())
print("Pajamų kvantiliai (25%, 50%, 75%):\n", revenue.quantile([0.25, 0.5, 0.75]).round(2))

In [ ]:
# describe pateikia greitą statistinę suvestinę
print(revenue.describe().round(2))

## 6. Kategorinių `Series` agregavimas (categorical series aggregation)

Kategoriniams duomenims dažniausiai naudojama:
- `value_counts()` – kiek kartų pasikartoja reikšmė
- `mode()` – dažniausia reikšmė (gali būti kelios)
- `nunique()` – unikalių reikšmių skaičius

Geroji praktika:
- naudoti `dropna=False`, kai svarbu matyti ir trūkstamas reikšmes
- aiškiai atskirti, kada naudojamas tekstas (kategorija), o kada skaičiai (metrika)

In [ ]:
cat_counts = category.value_counts()
print("Kategorijų dažniai:\n", cat_counts, "\n")
print("Unikalių kategorijų skaičius:", category.nunique())
print("Dažniausia kategorija (mode):\n", category.mode())

### Kategorija + metrika: paprasta suvestinė

Dažna analitinė užduotis: apskaičiuoti metriką pagal kategoriją.
`Series` atveju patogiausia naudoti `groupby` su agregavimu.

In [ ]:
# Pajamų suma pagal kategoriją
rev_by_cat = revenue.groupby(category).sum().sort_values(ascending=False)

print("Pajamų suma pagal kategoriją:\n", rev_by_cat.round(2))

In [ ]:
# Vidutinė kaina pagal kategoriją
avg_price_by_cat = price.groupby(category).mean().sort_values(ascending=False)

print("Vidutinė kaina pagal kategoriją:\n", avg_price_by_cat.round(2))

## 7. Dažnos klaidos ir gerosios praktikos (santrauka)

Dažnos klaidos:
- `and` / `or` naudojimas su `Series` vietoje `&` / `|`.
- Skliaustų nenaudojimas sudėtinėse sąlygose.
- Rikiavimas, kai skaitiniai duomenys yra `object` tipo (rikiuojama kaip tekstas).
- Nesutampantys indeksai aritmetikoje, kai atsiranda `NaN`, o tai neteisingai interpretuojama kaip klaida.

Gerosios praktikos:
- Naudoti `between` ir `isin` aiškesniam filtravimui.
- Aritmetikai naudoti metodus (`add`, `mul`, ...) kai reikia `fill_value` ar aiškesnio valdymo.
- Tekstui naudoti `.str` ir `na=False`, kai yra trūkstamų reikšmių.
- Prieš lyginimą ir rikiavimą pasitikrinti `dtype` ir, jei reikia, konvertuoti.